In [1]:
!pip install arch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.4/916.4 kB 45.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf
import arch

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import os

Mounted at /content/drive


In [4]:
os.chdir('/content/drive/My Drive/thesis/final_submission')
df = pd.read_csv('non_nan_4.csv')

In [5]:
df

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2000-02-01,4.1,170.000,6.661000,0.496560,-1.522563,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,2.220031,3.838386,-11.226228,54.440789,1.000,-0.1,-0.141500
1,2000-03-01,4.0,171.000,6.519500,0.511258,9.413333,22.264136,-0.218329,-2.079067,5.804243,0.913712,8.390897,6.420237,14.101954,6.922258,1.000,-0.1,-0.141500
2,2000-04-01,3.8,170.900,6.256522,1.327803,-3.266805,5.567379,-8.205683,17.437698,23.153694,12.400712,-0.097663,2.559423,15.213674,7.370518,-0.100,-0.2,-0.262978
3,2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,7.374072,20.863985,5.169310,5.638019,-8.163265,0.300,0.2,-0.265995
4,2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,3.078671,2.813690,8.474599,8.076012,13.131313,1.000,0.0,0.449928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
276,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
277,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
278,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [6]:
def create_lags_cov(data, covs, period):
    lagged_covs = []  # List to store the names of lagged columns

    for covariate in covs:
        for i in period:
            lagged_column_name = f'{covariate}_lag{i}'
            data[lagged_column_name] = data[covariate].shift(i)
            lagged_covs.append(lagged_column_name)

    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_covs

In [7]:
def create_lags_returns(data, tickers, period):

    lagged_tickers = []  # List to store the names of lagged columns

    for tick in tickers:
        for i in period:
            lagged_column_name = f'{tick}_lag{i}'
            data[lagged_column_name] = data[tick].shift(i)
            lagged_tickers.append(lagged_column_name)

    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_tickers

In [8]:
# Creating lagged variables
tickers = ['AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period_covs = [1, 2, 3]
period_returns = [1,2]

data, lagged_covs = create_lags_cov(df, covs, period=period_covs)
data, lagged_tickers = create_lags_returns(df, tickers, period= period_returns)

In [10]:
# Creating dummy variables
data['Quater_dummy'] = 0
data['Quater_rippel'] = 0
data['Date'] = pd.to_datetime(data['Date'])
for index, row in data.iterrows():
    if row['Date'].month in [3, 6, 9, 12]:
        data.loc[index, 'Quater_dummy'] = 1
    if row['Date'].month in [1, 4, 7, 10]:
        data.loc[index, 'Quater_dummy'] = 1

In [11]:
data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,...,NVO_lag1,NVO_lag2,NVS_lag1,NVS_lag2,PFE_lag1,PFE_lag2,ROG_lag1,ROG_lag2,Quater_dummy,Quater_rippel
0,2000-07-01,4.0,172.700,6.097273,-0.653855,-1.338453,-8.064479,-11.373392,-8.650271,4.005041,...,2.813690,20.863985,8.474599,5.169310,8.076012,5.638019,13.131313,-8.163265,1,0
1,2000-08-01,4.1,172.700,6.054000,0.795710,6.534124,6.578935,3.131245,-1.192113,-29.708189,...,13.757382,2.813690,-3.593737,8.474599,-9.635421,8.076012,2.142857,13.131313,0,0
2,2000-09-01,3.9,173.600,5.826087,0.249670,-5.723123,15.363525,8.458631,2.496165,11.482185,...,5.591665,13.757382,-1.944901,-3.593737,-0.540360,-9.635421,-2.447552,2.142857,1,0
3,2000-10-01,3.9,173.900,5.799000,0.187218,-0.217758,-8.764862,5.978226,-1.929509,10.169493,...,9.359631,5.591665,2.314016,-1.944901,4.352595,-0.540360,-9.318996,-2.447552,1,0
4,2000-11-01,3.9,174.200,5.738571,0.009919,-7.465269,7.863717,14.237843,8.548194,4.825204,...,-4.279274,9.359631,-0.807726,2.314016,-3.860903,4.352595,13.043478,-9.318996,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,...,8.619586,14.479974,1.295221,10.390734,3.090302,7.690670,9.446071,-53.665066,1,0
271,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,...,2.541749,8.619586,-0.110227,1.295221,-13.817335,3.090302,16.968326,9.446071,0,0
272,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,...,1.592445,2.541749,-7.172811,-0.110227,-7.286115,-13.817335,5.451681,16.968326,1,0
273,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,...,12.873250,1.592445,9.367574,-7.172811,0.566924,-7.286115,11.025813,5.451681,1,0


In [ ]:
"""def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss, df"""

In [12]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss

# 1 MONTH AHEAD:

In [13]:
import pandas as pd
import statsmodels.api as sm
from arch import arch_model
from scipy.stats import norm

def rolling_1m_forecast(data, dates, ticker, alpha):
    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=[ticker, 'Date'])
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=[ticker, 'Date'])
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=1)
        cond_vol_forecast = forecasts.variance.iloc[-1].apply(lambda x: np.sqrt(x))
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)

        loss = tick_loss(alpha, y_test.iloc[0], VaR_forecast)

        t_loss += loss

    t_loss = t_loss / 12

    return t_loss

In [14]:
dates1m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01']

In [15]:
for t in tickers:
    loss = rolling_1m_forecast(data, dates1m, t, 0.05)
    print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      4,   Func. Count:     28,   Neg. LLF: 889.7749000463948
Iteration:      5,   Func. Count:     33,   Neg. LLF: 889.7675789803586
Iteration:      6,   Func. Count:     38,   Neg. LLF: 889.7537238994715
Iteration:      7,   Func. Count:     43,   Neg. LLF: 889.7361309667915
Iteration:      8,   Func. Count:     48,   Neg. LLF: 889.7274260931832
Iteration:      9,   Func. Count:     53,   Neg. LLF: 889.725416391847
Iteration:     10,   Func. Count:     58,   Neg. LLF: 889.7253145862583
Iteration:     11,   Func. Count:     62,   Neg. LLF: 889.7253145862426
Optimization terminated successfully    (Exit mode 0)
            Current function value: 889.7253145862583
            Iterations: 11
            Function evaluations: 62
            Gradient evaluations: 11
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1239.3168332674327
Iteration:      2,   Func. Count:     13,   Neg. LLF: 897.2436887256465
Iteration:      3,   Func. Count:     19,   Neg. LLF: 896.728928670834


/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:     10,   Func. Count:     56,   Neg. LLF: 885.2128950597746
Iteration:     11,   Func. Count:     61,   Neg. LLF: 885.2127609855501
Iteration:     12,   Func. Count:     66,   Neg. LLF: 885.2127585731427
Iteration:     13,   Func. Count:     70,   Neg. LLF: 885.2127585730582
Optimization terminated successfully    (Exit mode 0)
            Current function value: 885.2127585731427
            Iterations: 13
            Function evaluations: 70
            Gradient evaluations: 13
Iteration:      1,   Func. Count:      6,   Neg. LLF: 906.9964907219263
Iteration:      2,   Func. Count:     13,   Neg. LLF: 913.222709191811
Iteration:      3,   Func. Count:     19,   Neg. LLF: 904.0742102389393
Iteration:      4,   Func. Count:     26,   Neg. LLF: 888.0176227402903
Iteration:      5,   Func. Count:     31,   Neg. LLF: 888.0170831290263
Iteration:      6,   Func. Count:     36,   Neg. LLF: 888.0170406502615
Iteration:      7,   Func. Count:     41,   Neg. LLF: 888.0167817047684


...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1093.690437547363
Iteration:      2,   Func. Count:     13,   Neg. LLF: 926.1642700173486
Iteration:      3,   Func. Count:     20,   Neg. LLF: 888.5973489007422
Iteration:      4,   Func. Count:     28,   Neg. LLF: 870.6959969513883
Iteration:      5,   Func. Count:     33,   Neg. LLF: 870.5420275978481
Iteration:      6,   Func. Count:     38,   Neg. LLF: 870.5424550033115
Iteration:      7,   Func. Count:     44,   Neg. LLF: 870.5409567053982
Iteration:      8,   Func. Count:     49,   Neg. LLF: 870.5409301801454
Iteration:      9,   Func. Count:     54,   Neg. LLF: 870.5409287572169
Iteration:     10,   Func. Count:     59,   Neg. LLF: 870.5409281504196
Optimization terminated successfully    (Exit mode 0)
            Current function value: 870.5409281504196
            Iterations: 10
            Function evaluations: 59
            Gradient evaluations: 10
PFE: 0.7735638737687962
Iteration:      1,   Func. Count:      6,   Neg

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      4,   Func. Count:     24,   Neg. LLF: 1026.5334575056986
Iteration:      5,   Func. Count:     30,   Neg. LLF: 1026.5321650488156
Iteration:      6,   Func. Count:     35,   Neg. LLF: 1026.5321236507464
Iteration:      7,   Func. Count:     40,   Neg. LLF: 1026.531915967955
Iteration:      8,   Func. Count:     45,   Neg. LLF: 1026.5310091930328
Iteration:      9,   Func. Count:     50,   Neg. LLF: 1026.5286922601586
Iteration:     10,   Func. Count:     55,   Neg. LLF: 1026.5282609248545
Iteration:     11,   Func. Count:     60,   Neg. LLF: 1026.5282287876234
Iteration:     12,   Func. Count:     65,   Neg. LLF: 1026.528228302634
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1026.528228302634
            Iterations: 12
            Function evaluations: 65
            Gradient evaluations: 12
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1156.7999856927881
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1075.3769

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

# 3 MONTHS AHEAD:

In [16]:
import pandas as pd
import statsmodels.api as sm
from arch import arch_model
from scipy.stats import norm

def rolling_3m_forecast(data, dates, ticker, alpha):
    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=[ticker, 'Date'])
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=[ticker, 'Date'])
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=3)  # Forecast for 3 months ahead
        cond_vol_forecast = forecasts.variance.iloc[-1].apply(lambda x: np.sqrt(x))
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)

        loss = tick_loss(alpha, y_test.iloc[0], VaR_forecast)

        t_loss += loss

    t_loss = t_loss / 10  # Adjusted for 10 months (since 'dates3m' has 10 elements)

    return t_loss

In [17]:
dates3m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01']

In [18]:
for t in tickers:
    loss = rolling_3m_forecast(data, dates3m, t, 0.05)
    print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      3,   Func. Count:     19,   Neg. LLF: 888.0315247874007
Iteration:      4,   Func. Count:     26,   Neg. LLF: 884.6907414212512
Iteration:      5,   Func. Count:     31,   Neg. LLF: 884.6848259256694
Iteration:      6,   Func. Count:     36,   Neg. LLF: 884.6574834856478
Iteration:      7,   Func. Count:     41,   Neg. LLF: 884.6397555584947
Iteration:      8,   Func. Count:     46,   Neg. LLF: 884.6286696410642
Iteration:      9,   Func. Count:     51,   Neg. LLF: 884.6273420337402
Iteration:     10,   Func. Count:     56,   Neg. LLF: 884.6273335106139
Iteration:     11,   Func. Count:     60,   Neg. LLF: 884.6273335108109
Optimization terminated successfully    (Exit mode 0)
            Current function value: 884.6273335106139
            Iterations: 11
            Function evaluations: 60
            Gradient evaluations: 11
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1178.7314788173323
Iteration:      2,   Func. Count:     14,   Neg. LLF: 893.966272220610

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      3,   Func. Count:     19,   Neg. LLF: 901.9676011216195
Iteration:      4,   Func. Count:     26,   Neg. LLF: 885.2200651421106
Iteration:      5,   Func. Count:     31,   Neg. LLF: 885.2195442810739
Iteration:      6,   Func. Count:     36,   Neg. LLF: 885.2195104425602
Iteration:      7,   Func. Count:     41,   Neg. LLF: 885.2193138255832
Iteration:      8,   Func. Count:     46,   Neg. LLF: 885.2183522300605
Iteration:      9,   Func. Count:     51,   Neg. LLF: 885.2147616595502
Iteration:     10,   Func. Count:     56,   Neg. LLF: 885.2128950597746
Iteration:     11,   Func. Count:     61,   Neg. LLF: 885.2127609855501
Iteration:     12,   Func. Count:     66,   Neg. LLF: 885.2127585731427
Iteration:     13,   Func. Count:     70,   Neg. LLF: 885.2127585730582
Optimization terminated successfully    (Exit mode 0)
            Current function value: 885.2127585731427
            Iterations: 13
            Function evaluations: 70
            Gradient evaluations: 13

...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1228.9085794123553
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1025.2056303614033
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1023.1943328513626
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1023.1942387003072
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1023.1941278027645
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1023.1941125884837
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1023.1940196342322
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1023.1938083986237
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1023.1934615938222
Iteration:     10,   Func. Count:     54,   Neg. LLF: 1023.1930778873815
Iteration:     11,   Func. Count:     59,   Neg. LLF: 1023.1928755673607
Iteration:     12,   Func. Count:     64,   Neg. LLF: 1023.1928461598402
Iteration:     13,   Func. Count:     69,   Neg. LLF: 1023.1928437372644
Iteration:     14,   Func. Count:     73,   Neg. LL

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1165.6506883972468
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1108.8767328283766
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1049.122615254116
Iteration:      4,   Func. Count:     25,   Neg. LLF: 1053.8626101087798
Iteration:      5,   Func. Count:     31,   Neg. LLF: 1056.5199787652168
Iteration:      6,   Func. Count:     37,   Neg. LLF: 1051.8992846478177
Iteration:      7,   Func. Count:     43,   Neg. LLF: 1050.2432814106528
Iteration:      8,   Func. Count:     49,   Neg. LLF: 1049.4173748869766
Iteration:      9,   Func. Count:     55,   Neg. LLF: 1048.733819232443
Iteration:     10,   Func. Count:     61,   Neg. LLF: 1048.6128017553729
Iteration:     11,   Func. Count:     66,   Neg. LLF: 1048.6103492391446
Iteration:     12,   Func. Count:     71,   Neg. LLF: 1048.6101817179322
Iteration:     13,   Func. Count:     76,   Neg. LLF: 1048.6100240024266
Iteration:     14,   Func. Count:     81,   Neg. LLF:

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


# 6 MONTHS AHEAD:

In [19]:
import pandas as pd
import statsmodels.api as sm
from arch import arch_model
from scipy.stats import norm

def rolling_6m_forecast(data, dates, ticker, alpha):
    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=[ticker, 'Date'])
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=[ticker, 'Date'])
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=6)  # Forecast for 6 months ahead
        cond_vol_forecast = forecasts.variance.iloc[-1].apply(lambda x: np.sqrt(x))
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)

        loss = tick_loss(alpha, y_test.iloc[0], VaR_forecast)

        t_loss += loss

    t_loss = t_loss / 7  # Adjusted for 7 months (since 'dates6m' has 7 elements)

    return t_loss

In [20]:
dates6m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01']

In [21]:
for t in tickers:
    loss = rolling_6m_forecast(data, dates6m, t, 0.05)
    print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1227.934884202285
Iteration:      2,   Func. Count:     13,   Neg. LLF: 889.2892608331276
Iteration:      3,   Func. Count:     19,   Neg. LLF: 888.0315247874007
Iteration:      4,   Func. Count:     26,   Neg. LLF: 884.6907414212512
Iteration:      5,   Func. Count:     31,   Neg. LLF: 884.6848259256694
Iteration:      6,   Func. Count:     36,   Neg. LLF: 884.6574834856478
Iteration:      7,   Func. Count:     41,   Neg. LLF: 884.6397555584947
Iteration:      8,   Func. Count:     46,   Neg. LLF: 884.6286696410642
Iteration:      9,   Func. Count:     51,   Neg. LLF: 884.6273420337402
Iteration:     10,   Func. Count:     56,   Neg. LLF: 884.6273335106139
Iteration:     11,   Func. Count:     60,   Neg. LLF: 884.6273335108109
Optimization terminated successfully    (Exit mode 0)
            Current function value: 884.6273335106139
            Iterations: 11
            Function evaluations: 60
            Gradient evaluations: 11

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      4,   Func. Count:     26,   Neg. LLF: 891.0219297504891
Iteration:      5,   Func. Count:     31,   Neg. LLF: 891.0213654327135
Iteration:      6,   Func. Count:     36,   Neg. LLF: 891.0213275042972
Iteration:      7,   Func. Count:     41,   Neg. LLF: 891.0210963483075
Iteration:      8,   Func. Count:     46,   Neg. LLF: 891.0199400989616
Iteration:      9,   Func. Count:     51,   Neg. LLF: 891.0159586622088
Iteration:     10,   Func. Count:     56,   Neg. LLF: 891.0147657259877
Iteration:     11,   Func. Count:     61,   Neg. LLF: 891.014698196681
Iteration:     12,   Func. Count:     66,   Neg. LLF: 891.0146939301426
Iteration:     13,   Func. Count:     70,   Neg. LLF: 891.0146939300045
Optimization terminated successfully    (Exit mode 0)
            Current function value: 891.0146939301426
            Iterations: 13
            Function evaluations: 70
            Gradient evaluations: 13
Iteration:      1,   Func. Count:      6,   Neg. LLF: 913.5615952448607


...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1058.3131753399266
Iteration:      2,   Func. Count:     13,   Neg. LLF: 857.7749864560318
Iteration:      3,   Func. Count:     20,   Neg. LLF: 962.5766427766032
Iteration:      4,   Func. Count:     27,   Neg. LLF: 843.349449512671
Iteration:      5,   Func. Count:     32,   Neg. LLF: 843.2718780137172
Iteration:      6,   Func. Count:     37,   Neg. LLF: 843.368911001076
Iteration:      7,   Func. Count:     43,   Neg. LLF: 843.2570399235453
Iteration:      8,   Func. Count:     48,   Neg. LLF: 843.256746432
Iteration:      9,   Func. Count:     53,   Neg. LLF: 843.2567433992162
Iteration:     10,   Func. Count:     57,   Neg. LLF: 843.256743398649
Optimization terminated successfully    (Exit mode 0)
            Current function value: 843.2567433992162
            Iterations: 10
            Function evaluations: 57
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1062.3470494938533
Iter

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter


Iteration:      3,   Func. Count:     19,   Neg. LLF: 1009.5684780724663
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1009.5683778425991
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1009.5682735267333
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1009.5680105828636
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1009.5663903221539
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1009.5628278473197
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1009.562807759047
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1009.5628077585338
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1009.562807759047
            Iterations: 10
            Function evaluations: 53
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1215.0219667989172
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1014.7641046057527
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1012.91

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

# 9 MONTHS AHEAD:

In [22]:
import pandas as pd
import statsmodels.api as sm
from arch import arch_model
from scipy.stats import norm

def rolling_9m_forecast(data, dates, ticker, alpha):
    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=[ticker, 'Date'])
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=[ticker, 'Date'])
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=9)  # Forecast for 9 months ahead
        cond_vol_forecast = forecasts.variance.iloc[-1].apply(lambda x: np.sqrt(x))
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)

        loss = tick_loss(alpha, y_test.iloc[0], VaR_forecast)

        t_loss += loss

    t_loss = t_loss / 4  # Adjusted for 4 months (since 'dates9m' has 4 elements)

    return t_loss

In [23]:
dates9m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01']

In [24]:
for t in tickers:
    loss = rolling_9m_forecast(data, dates9m, t, 0.05)
    print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      3,   Func. Count:     19,   Neg. LLF: 901.9676011216195
Iteration:      4,   Func. Count:     26,   Neg. LLF: 885.2200651421106
Iteration:      5,   Func. Count:     31,   Neg. LLF: 885.2195442810739
Iteration:      6,   Func. Count:     36,   Neg. LLF: 885.2195104425602
Iteration:      7,   Func. Count:     41,   Neg. LLF: 885.2193138255832
Iteration:      8,   Func. Count:     46,   Neg. LLF: 885.2183522300605
Iteration:      9,   Func. Count:     51,   Neg. LLF: 885.2147616595502
Iteration:     10,   Func. Count:     56,   Neg. LLF: 885.2128950597746
Iteration:     11,   Func. Count:     61,   Neg. LLF: 885.2127609855501
Iteration:     12,   Func. Count:     66,   Neg. LLF: 885.2127585731427
Iteration:     13,   Func. Count:     70,   Neg. LLF: 885.2127585730582
Optimization terminated successfully    (Exit mode 0)
            Current function value: 885.2127585731427
            Iterations: 13
            Function evaluations: 70
            Gradient evaluations: 13

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      9,   Func. Count:     51,   Neg. LLF: 894.8595683597949
Iteration:     10,   Func. Count:     56,   Neg. LLF: 894.8576466558918
Iteration:     11,   Func. Count:     61,   Neg. LLF: 894.8574935856548
Iteration:     12,   Func. Count:     66,   Neg. LLF: 894.8574870038208
Iteration:     13,   Func. Count:     71,   Neg. LLF: 894.8574862570197
Optimization terminated successfully    (Exit mode 0)
            Current function value: 894.8574862570197
            Iterations: 13
            Function evaluations: 71
            Gradient evaluations: 13
BMY: 4.273425518571841
Iteration:      1,   Func. Count:      6,   Neg. LLF: 3737.673074947868
Iteration:      2,   Func. Count:     14,   Neg. LLF: 1059.3458407887865
Iteration:      3,   Func. Count:     20,   Neg. LLF: 990.3442313922687
Iteration:      4,   Func. Count:     26,   Neg. LLF: 774.7400448569347
Iteration:      5,   Func. Count:     32,   Neg. LLF: 779.060699049972
Iteration:      6,   Func. Count:     38,   Neg.

...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1058.3131753399266
Iteration:      2,   Func. Count:     13,   Neg. LLF: 857.7749864560318
Iteration:      3,   Func. Count:     20,   Neg. LLF: 962.5766427766032
Iteration:      4,   Func. Count:     27,   Neg. LLF: 843.349449512671
Iteration:      5,   Func. Count:     32,   Neg. LLF: 843.2718780137172
Iteration:      6,   Func. Count:     37,   Neg. LLF: 843.368911001076
Iteration:      7,   Func. Count:     43,   Neg. LLF: 843.2570399235453
Iteration:      8,   Func. Count:     48,   Neg. LLF: 843.256746432
Iteration:      9,   Func. Count:     53,   Neg. LLF: 843.2567433992162
Iteration:     10,   Func. Count:     57,   Neg. LLF: 843.256743398649
Optimization terminated successfully    (Exit mode 0)
            Current function value: 843.2567433992162
            Iterations: 10
            Function evaluations: 57
            Gradient evaluations: 10
PFE: 3.4260101010202124
Iteration:      1,   Func. Count:      6,   Neg. LLF:

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1224.3227681527878
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1021.7568942469534
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1019.8138887813601
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1019.8138548991371
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1019.8138374640391
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1019.8138352299792
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1019.8138339417283
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1019.8138256839918
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1019.8137889992914
Iteration:     10,   Func. Count:     54,   Neg. LLF: 1019.8137575943713
Iteration:     11,   Func. Count:     59,   Neg. LLF: 1019.8137503012683
Iteration:     12,   Func. Count:     64,   Neg. LLF: 1019.8137489296873
Iteration:     13,   Func. Count:     68,   Neg. LLF: 1019.813748930081
Optimization terminated successfully    (Exit mode 0

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


# 12 MONTHS AHEAD:

In [25]:
import pandas as pd
import statsmodels.api as sm
from arch import arch_model
from scipy.stats import norm

def rolling_12m_forecast(data, dates, ticker, alpha):
    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=[ticker, 'Date'])
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=[ticker, 'Date'])
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=12)  # Forecast for 12 months ahead
        cond_vol_forecast = forecasts.variance.iloc[-1].apply(lambda x: np.sqrt(x))
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)

        loss = tick_loss(alpha, y_test.iloc[0], VaR_forecast)

        t_loss += loss

    t_loss = t_loss / len(dates)  # Adjusted for the number of months in 'dates12m'

    return t_loss

In [26]:
dates12m = ['2022-05-01']

In [27]:
for t in tickers:
    loss = rolling_12m_forecast(data, dates12m, t, 0.05)
    print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
AZ

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

LLY: 8.914244012274354
Iteration:      1,   Func. Count:      6,   Neg. LLF: 8317.48338850205
Iteration:      2,   Func. Count:     15,   Neg. LLF: 1027.937421865165
Iteration:      3,   Func. Count:     22,   Neg. LLF: 874.724511625416
Iteration:      4,   Func. Count:     28,   Neg. LLF: 869.9097933437865
Iteration:      5,   Func. Count:     34,   Neg. LLF: 868.2991652649121
Iteration:      6,   Func. Count:     40,   Neg. LLF: 867.9631910233925
Iteration:      7,   Func. Count:     45,   Neg. LLF: 867.9613124582204
Iteration:      8,   Func. Count:     50,   Neg. LLF: 867.9613203590475
Iteration:      9,   Func. Count:     56,   Neg. LLF: 867.9612489980823
Iteration:     10,   Func. Count:     60,   Neg. LLF: 867.9612489969106
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.9612489980823
            Iterations: 10
            Function evaluations: 60
            Gradient evaluations: 10
MRK: 5.2504360367705525
Iteration:      1,   Func.

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

PFE: 7.323187380168994
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1210.4827289658347
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1011.3744844764094
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1009.5684780724663
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1009.5683778425991
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1009.5682735267333
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1009.5680105828636
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1009.5663903221539
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1009.5628278473197
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1009.562807759047
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1009.5628077585338
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1009.562807759047
            Iterations: 10
            Function evaluations: 53
            Gradient evaluations: 10
ROG: 9.489380154529998


/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
